<a href="https://colab.research.google.com/github/e-olang/NLP/blob/main/Q%26A/QA_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("masakhane/afriqa", "swa")
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 415
    })
    validation: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 417
    })
    test: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 302
    })
})

In [ ]:
dataset['train']['question'][:5]

['Je,mkoa wa Arusha ina idadi ya watu wangapi?',
 'Je,Thiago Silva alichezea Brazil katika kombe la dunia mwaka 2018?',
 'Sun Yat-sen alikuwa ameoa?',
 'Je,Wamaragoli huzungumza lugha gani?',
 'Gavana wa kwanza kaunti ya Marsabit ni nani?']

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("eolang/SW-v1")

In [ ]:
dataset["train"].column_names

['question',
 'answers',
 'lang',
 'split',
 'translated_question',
 'translated_answer',
 'translation_type']

In [ ]:
['question',
 'answers'
 ]